In [1]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx
import math

In [2]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/sax.wav"

In [19]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([ 0.0053,  0.0062,  0.0075,  ..., -0.0001, -0.0001, -0.0002])


In [20]:
audio.shape

torch.Size([57618])

In [21]:
temp_input = torch.rand(16)
temp_input

tensor([0.6683, 0.4344, 0.3136, 0.0229, 0.7162, 0.8960, 0.8064, 0.6667, 0.1974,
        0.3597, 0.2674, 0.8992, 0.7366, 0.9646, 0.9806, 0.9079])

In [7]:
rfft_audio.apply_rfft(temp_input)

tensor([8.2707, 1.7070, 0.9202, 0.6153, 0.6040, 1.1199, 1.2618, 1.1066, 0.3693])

In [8]:
def fft(samples):
    N = len(samples)

    if N == 1:
        return samples
    
    M = N // 2

    Xeven = [0. for i in range(M)]
    Xodd = [0. for i in range(M)]

    for i in range(M):
        Xeven[i] = samples[2 * i]
        Xodd[i] = samples[2 * i + 1]
    
    Feven = [0. for i in range(M)]
    Feven = fft(Xeven)

    Fodd = [0. for i in range(M)]
    Fodd = fft(Xodd)

    freqbin = [0. for i in range(N)]

    for k in range(N // 2):
        cmplx = complex(1,-2 * math.pi * k / N) * Fodd[k]

        freqbin[k] = Feven[k] + cmplx
        freqbin[k + N // 2] = Feven[k] - cmplx

    return abs(torch.tensor(freqbin))

In [9]:
ans = fft(temp_input)

In [10]:
abs(torch.fft.fft(temp_input))

tensor([8.2707, 1.7070, 0.9202, 0.6153, 0.6040, 1.1199, 1.2618, 1.1066, 0.3693,
        1.1066, 1.2618, 1.1199, 0.6040, 0.6153, 0.9202, 1.7070])

In [11]:
abs(torch.tensor(ans))

/tmp/ipykernel_235996/2932582670.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  abs(torch.tensor(ans))


tensor([8.2707, 4.1898, 1.6523, 6.6004, 0.8716, 3.1958, 1.4449, 8.7046, 0.3693,
        1.1924, 0.9558, 3.7811, 0.4891, 2.0737, 0.9641, 7.2168])

In [12]:
temp_input.shape[0]

16

In [11]:
import torch
import math

def next_power_of_2_calc(n):
    if n <= 1:
        return 1
    return 2 ** (math.ceil(math.log2(n)))

def fft(samples:torch.Tensor):

    N = samples.shape[0]
    if (N & (N - 1)) != 0:
        next_power_of_2 = next_power_of_2_calc(N)  # Compute the next power of 2
        padding_size = next_power_of_2 - N
        samples = torch.cat([samples, torch.zeros(padding_size)])
        N = next_power_of_2
    
    if N == 1:
        return samples
    
    even_samples = samples[::2]
    odd_samples = samples[1::2]
    
    Feven = fft(even_samples)
    Fodd = fft(odd_samples)
    
    omega = torch.exp(-2j * math.pi * torch.arange(N // 2) / N) 
    combined = torch.zeros(N, dtype=torch.complex64)
    
    combined[:N//2] = Feven + omega * Fodd
    combined[N//2:] = Feven - omega * Fodd

    return combined


In [15]:
import torch
import math

def next_power_of_2_calc(n):
    if n <= 1:
        return 1
    return 2 ** (math.ceil(math.log2(n)))

def fft_real_imag(samples: torch.Tensor):
    N = samples.shape[0]
    
    # Pad to the next power of 2 if necessary
    if (N & (N - 1)) != 0:
        next_power_of_2 = next_power_of_2_calc(N)
        padding_size = next_power_of_2 - N
        samples = torch.cat([samples, torch.zeros(padding_size)])
        N = next_power_of_2
    
    if N == 1:
        return samples, torch.zeros_like(samples)
    
    even_samples = samples[::2]
    odd_samples = samples[1::2]
    
    Feven_real, Feven_imag = fft_real_imag(even_samples)
    Fodd_real, Fodd_imag = fft_real_imag(odd_samples)
    
    angle = -2j * math.pi * torch.arange(N // 2) / N
    omega_real = torch.cos(angle)
    omega_imag = torch.sin(angle)
    
    # Combine the real and imaginary parts manually
    combined_real = torch.zeros(N)
    combined_imag = torch.zeros(N)
    
    combined_real[:N//2] = Feven_real + omega_real * Fodd_real - omega_imag * Fodd_imag
    combined_imag[:N//2] = Feven_imag + omega_real * Fodd_imag + omega_imag * Fodd_real
    
    combined_real[N//2:] = Feven_real - omega_real * Fodd_real + omega_imag * Fodd_imag
    combined_imag[N//2:] = Feven_imag - omega_real * Fodd_imag - omega_imag * Fodd_real
    
    return combined_real, combined_imag

# Example usage with ONNX exportable output
samples = torch.randn(1024)
real_part, imag_part = fft_real_imag(samples)


/tmp/ipykernel_242397/1300399318.py:36: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /opt/conda/conda-bld/pytorch_1724788959732/work/aten/src/ATen/native/Copy.cpp:305.)
  combined_real[:N//2] = Feven_real + omega_real * Fodd_real - omega_imag * Fodd_imag


In [12]:
temp_input = torch.rand(1024*1024*256)

In [13]:
x = abs(torch.fft.fft(temp_input))

In [14]:
y = fft(temp_input)

KeyboardInterrupt: 

In [29]:
torch.mean(abs(x-y))

tensor(7.1705)